In [ ]:
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    "DataFrames",
    "uCSV"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [ ]:
project_dir = dirname(pwd())
data_dir = mkpath(joinpath(project_dir, "data"))

In [ ]:
viral_reference_fastas_dir = mkpath(joinpath(data_dir, "viral-reference-fastas"))

In [ ]:
reference_fastas = filter(x -> occursin(r"\.f(na|asta)\.gz$", x), readdir(viral_reference_fastas_dir, join=true))
sort!(reference_fastas, by=x->filesize(x))

In [ ]:
slurm_logdir = mkpath("$(homedir())/workspace/slurmlogs")

In [ ]:
# make sketches for references

# 7384.737911 seconds (338.18 k allocations: 15.754 MiB, 0.00% compilation time)
# 2 hours
# 
# 16107.376116 seconds (654.58 k allocations: 28.929 MiB, 0.00% gc time)
# 4 hours

# @time run(`$(Mycelia.MAMBA) run --live-stream -n sourmash sourmash sketch dna -p scaled=1000,k=31 $(reference_fasta) -o $(reference_fasta).sig`)

In [ ]:
# didn't finish in 24 hours
# scaled=1000,k=21,k=31,k=51
# didn't finish in 24 hours
# scaled=1000,k=17,k=23,k=31
# scaled=1000,k=17,k=23,k=31

for reference_fasta in [
    "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/IMGVR_all_nucleotides-high_confidence.fna.gz",
    "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/nt_viruses.fasta.gz"
]
    # 17
    for k in [3, 5, 7, 11, 13, 23, 31, 41, 53, 67]
        sourmash_sketch = reference_fasta * ".k$(k).sourmash-sketch.zip"
        cmd = "$(Mycelia.MAMBA) run --live-stream -n sourmash sourmash sketch dna -p scaled=1000,k=$(k) --singleton $(reference_fasta) -o $(sourmash_sketch)"
        Mycelia.sbatch(
            job_name = "$(basename(sourmash_sketch))",
            mail_user = "cameron.prybol@gmail.com",
            logdir = slurm_logdir,
            partition = "batch",
            account = "mpsnyder",
            mem_gb = 16,
            cpus_per_task = 2,
            cmd = cmd)
    end
end

In [ ]:
# # for sample in sample_directories[1:1]
# sample = sample_directories[1]
# forward = joinpath(sample, "trim_galore", "$(basename(sample))_1_val_1.fq.gz")
# reverse = joinpath(sample, "trim_galore", "$(basename(sample))_2_val_2.fq.gz")
# outdir = mkpath(joinpath(sample, "sourmash"))
# sample_id = basename(sample) * "." * basename(sourmash_sketch)
# file_name = sample_id * ".sourmash-sketch.zip"
# file_path = joinpath(outdir, file_name)
# #     # @show outfile
# #     if !isfile(file_path) || (filesize(file_path) == 0)
# #         cmd = `$(Mycelia.MAMBA) run --live-stream -n sourmash sourmash sketch dna -p k=31 $(forward) $(reverse) --name $(sample_id) -o $(file_path)`
# #         run(cmd)
# #         # """
# #         # $(Mycelia.MAMBA) run --live-stream -n bwa-mem2 bwa-mem2 mem -t $(threads) $(fasta_file) $(forward) $(reverse) \\
# #         # | $(Mycelia.MAMBA) run --live-stream -n samtools samtools sort -u --threads $(threads) \\
# #         # | $(Mycelia.MAMBA) run --live-stream -n samtools samtools view --bam --with-header --output $(outfile)
# #         # """
# #         # Mycelia.sbatch(
# #         #     job_name = "bwa-mem2.$(basename(fasta_file)).$(basename(sample))",
# #         #     mail_user = "cameron.prybol@gmail.com",
# #         #     logdir = slurm_logdir,
# #         #     partition = "batch",
# #         #     account = "mpsnyder",
# #         #     mem_gb = memory,
# #         #     cpus_per_task= threads,
# #         #     cmd = cmd)
# #     else
# #         @info "$(outfile) already present..."
# #     end
# # end

In [ ]:
@time run(`$(Mycelia.MAMBA) run --live-stream -n sourmash sourmash gather -o $(file_path).sourmash-gather.csv $(file_path) $(sourmash_sketch)`)

In [ ]:
# # sketch reads for each sample

# sourmash sketch dna -p k=21,k=31,k=51,abund metagenome.fq.gz

# sourmash sketch dna -p k=31 sample_R1.fq.gz sample_R2.fq.gz --name "sample" -o sample.zip

# gunzip -c sample_R?.fq.gz | sourmash sketch dna -p k=31 - \
#     -o sample.zip

In [ ]:
# # merge all of the read sketches
# sourmash signature merge file1.sig file2.sig -o merged.sig

In [ ]:
# gather each sample

In [ ]:
# sourmash gather -p abund -o $(query).$(reference).sourmash-gather.csv query.sig <database1> [ <database2> ... ]

In [ ]:
# gather the merged signature

In [ ]:
# review outputs

In [ ]:
# filter and then map reads if I want more detail